# Corte de fatores de risco para malformacoes cardiacas SINAC

In [1]:
from pysus import SINASC
import pandas as pd
import re

In [2]:
# Lista de anos desejados
sinasc = SINASC().load()
#UF = ['AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE']
anos = [2021,2022,2023]
files = sinasc.get_files("DN", year=anos)

In [3]:
parquets = sinasc.download(files)
dfs = [p.to_dataframe() for p in parquets]
df = pd.concat(dfs, ignore_index=True)

  0%|          | 0/55 [00:00<?, ?it/s]

1127918it [00:00, 133862397.19it/s]   


In [4]:
#print(df.columns)
colunas_numericas = ["TPNASCASSI","IDADEMAE", "ESTCIVMAE", "GRAVIDEZ","PESO", "SEMAGESTAC", "CONSULTAS","PARTO","CODOCUPMAE","QTDFILVIVO","QTDFILMORT"]
for col in colunas_numericas:
    df[col] = pd.to_numeric(df[col], errors='coerce')

print(f"Numero total de Nacidos vivos registrados: {len(df)}")

Numero total de Nacidos vivos registrados: 5239566


## Filtrar df

In [5]:
variaveis = ["IDADEMAE", "ESTCIVMAE", "GRAVIDEZ","PESO", "SEMAGESTAC", "CONSULTAS", "CODANOMAL","ESCMAE2010","PARTO","CODOCUPMAE","SEXO","QTDFILVIVO","QTDFILMORT"]
df_filtrado_variaveis = df[variaveis]

In [6]:
df =0

In [7]:
def filter_ignored_data(df):
    """
    Remove do DataFrame as linhas onde alguma das variáveis
    ESTCIVMAE, GRAVIDEZ, ESCMAE2010 ou PARTO tenha o valor 9.
    
    Parâmetros:
      df: DataFrame contendo as colunas mencionadas.
      
    Retorna:
      DataFrame filtrado.
    """
    mask = (
        (df['ESTCIVMAE'] != 9) &
        (df['GRAVIDEZ'] != 9) &
        (df['ESCMAE2010'] != 9) &
        (df['PARTO'] != 9) &
        (df['SEXO'] != 0)&
        (df['CONSULTAS'] !=9)&
        (df['QTDFILMORT'] != 99)&
        (df['CONSULTAS'] !=9)
        )
    df_filtrado = df[mask].copy()
    return df_filtrado


In [8]:
df_filtrado_variaveis = filter_ignored_data(df_filtrado_variaveis)

print(f"Numero total de Nacidos vivos registrados depois da exclusao de dados faltantes das variaveis SEXO, Tipo de PARTO, GRAVIDEZ,Consultas, EStado Civil e ESCMAE2010: {len(df_filtrado_variaveis)}")

Numero total de Nacidos vivos registrados depois da exclusao de dados faltantes das variaveis SEXO, Tipo de PARTO, GRAVIDEZ,Consultas, EStado Civil e ESCMAE2010: 5012623


#### Gerando Df da amostra

In [9]:
codigos_cid = [
    "Q03", #Q030 - hidrocefalia congenita
]

In [10]:
df_filtrado_variaveis.loc[:, 'CODANOMAL'] = (
    df_filtrado_variaveis['CODANOMAL']
    .astype(str)
    .str.strip()
    .str.upper()
)

# Exclua registros com dados ausentes em variáveis essenciais
df_filtrado_variaveis = df_filtrado_variaveis.dropna(subset=['IDADEMAE','SEMAGESTAC'])

# Crie uma expressão regular para identificar códigos cardíacos em qualquer posição
regex_cid = '(?:' + '|'.join(codigos_cid) + ')'

# Máscaras para identificar:
# - Registros sem anomalia: CODANOMAL é string vazia
#mask_sem_anomalia = (df_filtrado_variaveis['CODANOMAL'] == '')

# - Registros com malformação cardíaca: CODANOMAL inicia com um dos códigos definidos CASOS
mask_cardiacas = df_filtrado_variaveis['CODANOMAL'].str.contains(regex_cid, na=False)

# Crie o df_model incluindo todos os registros com informação de anomalia
df_model = df_filtrado_variaveis.copy()

# Crie a variável binária 'CARDIAC': 1 para malformação cardíaca, 0 caso contrário
df_model['CASO'] = 0
df_model.loc[mask_cardiacas, 'CASO'] = 1

# Crie a variável 'OTHER_MALF' para outras malformações (não cardíacas):
# Será 1 se houver algum código (não vazio) E esse código NÃO for de malformação cardíaca
#mask_other = (df_model['CODANOMAL'] != '') & (~mask_cardiacas)

#df_model['OTHER_MALF'] = 0
#df_model.loc[mask_other, 'OTHER_MALF'] = 1



# Aplicando as funções no DataFrame
#df_model['CARDIAC'] = df_model['CODANOMAL'].apply(lambda x: 1 if tem_cardiac(x, codigos_cid) else 0)
#df_model['OTHER_MALF'] = df_model['CODANOMAL'].apply(lambda x: 1 if tem_other(x, codigos_cid) else 0)

# Visualize a distribuição das variáveis
print("Distribuição de CASO:")
print(df_model['CASO'].value_counts())

Distribuição de CASO:
CASO
0    4969723
1       1047
Name: count, dtype: int64


In [13]:
df_filtrado_variaveis.loc[:, 'CODANOMAL'] = (
    df_filtrado_variaveis['CODANOMAL']
    .astype(str)
    .str.strip()
    .str.upper()
    .replace(['NAN', 'NaN', 'nan'], '')  # Remove representações de NaN
)

# 3. Função para extrair códigos válidos de uma string concatenada
def extrair_codigos(concatenado):
    # Regex para encontrar códigos CID-10 (ex: Q20, Q21, etc.)
    padrao_cid = r'Q\d{3}'  # Q seguido de 2 dígitos
    return re.findall(padrao_cid, concatenado)

# 4. Aplicar a função para extrair códigos e verificar se são cardíacos
mask_cardiacas = df_filtrado_variaveis['CODANOMAL'].apply(
    lambda s: any(codigo in codigos_cid for codigo in extrair_codigos(s))
)

# 5. Máscara para registros sem anomalia
mask_sem_anomalia = (df_filtrado_variaveis['CODANOMAL'] == '')

# 6. Máscara para outras malformações (não cardíacas e não vazias)
mask_other = (~mask_sem_anomalia) & (~mask_cardiacas)

# 7. Criar as variáveis no DataFrame
df_model = df_filtrado_variaveis.copy()
df_model['CASO'] = 0
df_model.loc[mask_cardiacas, 'CASO'] = 1

df_model['OTHER_MALF'] = 0
df_model.loc[mask_other, 'OTHER_MALF'] = 1

# 8. Visualizar a distribuição
print("Distribuição de CASO:")
print(df_model['CASO'].value_counts())
print("\nDistribuição de OTHER_MALF:")
print(df_model['OTHER_MALF'].value_counts())

Distribuição de CASO:
CASO
0    4639913
1        282
Name: count, dtype: int64

Distribuição de OTHER_MALF:
OTHER_MALF
0    4598703
1      41492
Name: count, dtype: int64


In [9]:
import re

# cópia e padronização
df = df_filtrado_variaveis.copy()
df['CODANOMAL'] = (
    df['CODANOMAL']
    .fillna('')
    .astype(str)
    .str.strip()
    .str.upper()
)

# regex para extrair códigos ICD: letra + 2 dígitos + opcional . + dígitos
pattern = r'[A-Z]\d{2}(?:\.\d+)?'

def extract_codes(s):
    # devolve lista de substrings que casam o padrão
    return re.findall(pattern, s)

def is_Q03_case(s):
    codes = extract_codes(s)
    # verifica se algum código começa exatamente com 'Q03'
    return any(code.startswith('Q03') for code in codes)

# aplica e cria variável binária
df['CASO'] = df['CODANOMAL'].apply(is_Q03_case).astype(int)

# distribuições
print(df['CASO'].value_counts())

CASO
0    5011575
1       1048
Name: count, dtype: int64


In [11]:
cases = df_model[df_model['CASO'] == 1]

In [19]:
def robust_frequency_matching(df, ratio=4, category_bins=None, return_summary=True):
    """
    Realiza matching por frequência de forma robusta utilizando as variáveis 'IDADEMAE' e 'SEMAGESTAC'.
    
    Parâmetros:
      - df: DataFrame que contenha as colunas:
            'CARDIAC' (1 para caso e 0 para controle),
            'IDADEMAE' e 'SEMAGESTAC'.
      - ratio: número desejado de controles por caso.
      - category_bins: dicionário com os limites dos bins para as variáveis.
           Exemplo: {'IDADEMAE': [10, 20, 30, 40, 50, 60], 'SEMAGESTAC': [28, 32, 37, 42, 50]}
      - return_summary: se True, a função retorna também um DataFrame resumo do matching por grupo.
      
    Retorna:
      - matched_df: DataFrame contendo os casos e os controles pareados.
      - summary_df (opcional): DataFrame com resumo do desempenho do matching em cada grupo.
    """

    df = df.copy()
    
    # Cria as colunas categóricas com base nos bins fornecidos
    if category_bins is not None:
        df['IDADEMAE_CAT'] = pd.cut(df['IDADEMAE'], bins=category_bins['IDADEMAE'], include_lowest=True)
        df['SEMAGESTAC_CAT'] = pd.cut(df['SEMAGESTAC'], bins=category_bins['SEMAGESTAC'], include_lowest=True)
    else:
        df['IDADEMAE_CAT'] = df['IDADEMAE']
        df['SEMAGESTAC_CAT'] = df['SEMAGESTAC']

    
    df = df.dropna(subset=['IDADEMAE_CAT', 'SEMAGESTAC_CAT'])
    
    # Separa casos e controles
    cases = df[df['CASO'] == 1].copy()
    controls = df[df['CASO'] == 0].copy()
    
    grouping_columns = ['IDADEMAE_CAT', 'SEMAGESTAC_CAT']
    # Agrupa os casos pelas combinações de categorias presentes nos dados
    case_groups = cases.groupby(grouping_columns, observed=True)
    
    selected_controls_indices = []
    summary = []  # Para armazenar estatísticas de matching por grupo
    
    for group_keys, group_cases in case_groups:
        # Seleciona os controles que pertencem ao mesmo grupo
        group_controls = controls[
            (controls['IDADEMAE_CAT'] == group_keys[0]) & 
            (controls['SEMAGESTAC_CAT'] == group_keys[1])
        ]
        
        n_cases = len(group_cases)
        required_controls = ratio * n_cases
        
        if len(group_controls) >= required_controls:
            selected = group_controls.sample(n=required_controls, random_state=42)
        else:
            print(f"Aviso: Grupo {group_keys} - necessários {required_controls} controles, disponíveis {len(group_controls)}.")
            selected = group_controls
        
        achieved_ratio = (len(selected) / n_cases) if n_cases > 0 else np.nan
        
        # Registra o resumo do grupo
        summary.append({
            'grupo': group_keys,
            'n_cases': n_cases,
            'required_controls': required_controls,
            'available_controls': len(group_controls),
            'selected_controls': len(selected),
            'achieved_ratio': achieved_ratio
        })
        
        selected_controls_indices.extend(selected.index.tolist())
        # Remove os controles já selecionados para evitar reuso
        controls = controls.drop(selected.index)
    
    matched_controls = df.loc[selected_controls_indices]
    matched_df = pd.concat([cases, matched_controls])
    
    if return_summary:
        summary_df = pd.DataFrame(summary)
        return matched_df, summary_df
    else:
        return matched_df

In [12]:
import pandas as pd
import numpy as np
import warnings
from typing import Optional, Dict, Tuple, Union

def robust_frequency_matching(
    df: pd.DataFrame,
    ratio: int = 4,
    category_bins: Optional[Dict[str, list]] = None,
    return_summary: bool = False,
    random_state: Optional[int] = 42
) -> Union[pd.DataFrame, Tuple[pd.DataFrame, pd.DataFrame]]:
    """
    Realiza matching por frequência de forma robusta utilizando as variáveis 'IDADEMAE' e 'SEMAGESTAC'.
    
    Parâmetros:
      - df: DataFrame que contenha as colunas:
            'CARDIAC' (1 para caso e 0 para controle),
            'IDADEMAE' e 'SEMAGESTAC'.
      - ratio: Número desejado de controles por caso.
      - category_bins: Dicionário com os limites dos bins para as variáveis.
           Exemplo: {'IDADEMAE': [10, 20, 30, 40, 50, 60], 'SEMAGESTAC': [28, 32, 37, 42, 50]}
      - return_summary: Se True, retorna também um DataFrame resumo do matching por grupo.
      - random_state: Semente para a amostragem aleatória.
      
    Retorna:
      - matched_df: DataFrame contendo os casos e os controles pareados.
      - summary_df (opcional): DataFrame com resumo do desempenho do matching em cada grupo.
    """
    # Verifica se as colunas necessárias estão presentes
    required_cols = {"CASO", "IDADEMAE", "SEMAGESTAC"}
    missing_cols = required_cols - set(df.columns)
    if missing_cols:
        raise ValueError(f"O DataFrame está faltando as seguintes colunas: {missing_cols}")

    df = df.copy()

    # Cria as colunas categóricas com base nos bins fornecidos
    if category_bins is not None:
        idade_bins = category_bins.get("IDADEMAE")
        semage_bins = category_bins.get("SEMAGESTAC")
        if idade_bins is None or semage_bins is None:
            raise ValueError("O dicionário category_bins deve conter as chaves 'IDADEMAE' e 'SEMAGESTAC'")
        df["IDADEMAE_CAT"] = pd.cut(df["IDADEMAE"], bins=idade_bins, include_lowest=True)
        df["SEMAGESTAC_CAT"] = pd.cut(df["SEMAGESTAC"], bins=semage_bins, include_lowest=True)
    else:
        df["IDADEMAE_CAT"] = df["IDADEMAE"]
        df["SEMAGESTAC_CAT"] = df["SEMAGESTAC"]

    # Remove linhas com valores ausentes nas variáveis de categoria
    df.dropna(subset=["IDADEMAE_CAT", "SEMAGESTAC_CAT"], inplace=True)

    # Separa casos e controles
    cases = df[df["CASO"] == 1].copy()
    controls = df[df["CASO"] == 0].copy()

    grouping_columns = ["IDADEMAE_CAT", "SEMAGESTAC_CAT"]

    summary_records = []      # Para armazenar o resumo do matching por grupo
    selected_controls_indices = []  # Índices dos controles selecionados

    # Agrupa os casos pelas combinações de categorias
    case_groups = cases.groupby(grouping_columns, observed=True)

    for group_keys, group_cases in case_groups:
        n_cases = len(group_cases)
        required_controls = ratio * n_cases

        # Seleciona os controles que pertencem ao mesmo grupo
        group_mask = (
            (controls["IDADEMAE_CAT"] == group_keys[0]) &
            (controls["SEMAGESTAC_CAT"] == group_keys[1])
        )
        group_controls = controls.loc[group_mask]

        if len(group_controls) >= required_controls:
            selected = group_controls.sample(n=required_controls, random_state=random_state)
        else:
            warnings.warn(
                f"Aviso: Grupo {group_keys} - necessários {required_controls} controles, disponíveis {len(group_controls)}."
            )
            selected = group_controls

        achieved_ratio = len(selected) / n_cases if n_cases > 0 else np.nan

        # Registra as estatísticas do grupo
        summary_records.append({
            "grupo": group_keys,
            "n_cases": n_cases,
            "required_controls": required_controls,
            "available_controls": len(group_controls),
            "selected_controls": len(selected),
            "achieved_ratio": achieved_ratio
        })

        selected_controls_indices.extend(selected.index.tolist())
        # Remove os controles já selecionados para evitar reuso
        controls = controls.drop(selected.index)

    matched_controls = df.loc[selected_controls_indices]
    matched_df = pd.concat([cases, matched_controls]).sort_index()

    if return_summary:
        summary_df = pd.DataFrame(summary_records)
        return matched_df, summary_df
    else:
        return matched_df


In [13]:
age_bins = [10,20,30,40,50,60]
gestation_bins = [22,24,27,31,36,41,60]
category_bins = {'IDADEMAE': age_bins, 'SEMAGESTAC': gestation_bins}

In [14]:
matched_df_= robust_frequency_matching(df_model, ratio=4, category_bins=category_bins)

In [15]:
print(matched_df_['CASO'].value_counts())

CASO
0    4184
1    1046
Name: count, dtype: int64


In [16]:
len(matched_df_)

5230

In [17]:
print(matched_df_['CODANOMAL'].value_counts())

CODANOMAL
                        4118
Q039                     473
Q038                      79
Q031                      49
Q039Q059                  28
                        ... 
Q039Q211Q423Q620           1
Q039Q068Q650Q661Q674       1
Q039Q040Q173Q210Q212       1
Q672Q771                   1
Q174Q385Q549               1
Name: count, Length: 384, dtype: Int64


In [18]:
matched_df_.to_csv("data_BR_Match.csv", encoding='utf-8')